#Features from application dataset

Этот блокнот создает признаки из наборов данных для обучения и тестирования application. Набор данных application является основным набором данных и включает в себя target: дефолт (target = 1) или отсутствие дефолта (target = 0) по кредиту. Целевая переменная доступна только для обучающих данных.

In [ ]:
import numpy as np
import pandas as pd

# -----------------------------------------------------
from google.colab import drive

# -----------------------------------------------------
import zipfile
import time
import sys
import os
import gc

In [ ]:
# Сброс ограничений на число столбцов
pd.set_option("display.max_columns", None)

# Сброс ограничений на число строк до 400
pd.set_option("display.max_rows", 400)

In [ ]:
# Подключение Google Drive к текущей среде выполнения

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Распаковка содержимого архива в текущую директорию

zip_file = "/content/drive/My Drive/home-credit-default-risk.zip"

z = zipfile.ZipFile(zip_file, "r")
z.extractall()

# печать списока файлов и папок из текущей директории
print(os.listdir())

['.config', 'drive', 'home-credit-default-risk', 'sample_data']


In [ ]:
# Чтение данных

application_train = pd.read_csv("/content/home-credit-default-risk/application_train.csv")
application_test = pd.read_csv("/content/home-credit-default-risk/application_test.csv")

homecredit_columns_description = pd.read_csv("/content/home-credit-default-risk/HomeCredit_columns_description.csv", encoding="ISO-8859-1")

In [ ]:
# Проверка

application.shape

In [ ]:
# Объединение данных обучения и тестирования

application_train["train"] = 1
application_test["train"] = 0
application = pd.concat([application_train, application_test])

application.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,train
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
# Описание столбцов в application
description_application = homecredit_columns_description[homecredit_columns_description["Table"] == "application_{train|test}.csv"]
for index, row in description_application.iterrows():
    print(row["Row"], row["Description"])

# Коррекция данных описания
homecredit_columns_description.loc[homecredit_columns_description["Row"] == "FLAG_EMP_PHONE", "Description"] = "Did client provide employer phone (1=YES, 0=NO)"
homecredit_columns_description.loc[homecredit_columns_description["Row"] == "FLAG_WORK_PHONE", "Description"] = "Did client provide work phone (1=YES, 0=NO)"

SK_ID_CURR ID of loan in our sample
TARGET Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)
NAME_CONTRACT_TYPE Identification if loan is cash or revolving
CODE_GENDER Gender of the client
FLAG_OWN_CAR Flag if the client owns a car
FLAG_OWN_REALTY Flag if client owns a house or flat
CNT_CHILDREN Number of children the client has
AMT_INCOME_TOTAL Income of the client
AMT_CREDIT Credit amount of the loan
AMT_ANNUITY Loan annuity
AMT_GOODS_PRICE For consumer loans it is the price of the goods for which the loan is given
NAME_TYPE_SUITE Who was accompanying client when he was applying for the loan
NAME_INCOME_TYPE Clients income type (businessman, working, maternity leave,)
NAME_EDUCATION_TYPE Level of highest education the client achieved
NAME_FAMILY_STATUS Family status of the client
NAME_HOUSING_TYPE What is the housing situation of the client (renti

In [ ]:
# Функция для отображения описания выбранного признака

def add_description(table, row, description):
    list_description = [np.NaN, table, row, description, np.NaN]
    homecredit_columns_description.loc[len(homecredit_columns_description)] = list_description
    return homecredit_columns_description

In [ ]:
# Замена очень редких наблюдений дискретными признаками
application.loc[application["CNT_CHILDREN"] >= 5, "CNT_CHILDREN"] = 5
application.loc[application["CODE_GENDER"] == "XNA", "CODE_GENDER"] = application["CODE_GENDER"].mode()
application.loc[application["NAME_FAMILY_STATUS"] == "Unknown", "NAME_FAMILY_STATUS"] = application["NAME_FAMILY_STATUS"].mode()
application.loc[application["CNT_FAM_MEMBERS"] >= 6, "CNT_FAM_MEMBERS"] = 6
application.loc[application["OBS_30_CNT_SOCIAL_CIRCLE"] >= 15, "OBS_30_CNT_SOCIAL_CIRCLE"] = 15
application.loc[application["DEF_30_CNT_SOCIAL_CIRCLE"] >= 5, "DEF_30_CNT_SOCIAL_CIRCLE"] = 5
application.loc[application["OBS_60_CNT_SOCIAL_CIRCLE"] >= 15, "OBS_60_CNT_SOCIAL_CIRCLE"] = 15
application.loc[application["DEF_60_CNT_SOCIAL_CIRCLE"] >= 5, "DEF_60_CNT_SOCIAL_CIRCLE"] = 5
application.loc[application["AMT_REQ_CREDIT_BUREAU_HOUR"] >= 2, "AMT_REQ_CREDIT_BUREAU_HOUR"] = 2
application.loc[application["AMT_REQ_CREDIT_BUREAU_DAY"] >= 3, "AMT_REQ_CREDIT_BUREAU_DAY"] = 3
application.loc[application["AMT_REQ_CREDIT_BUREAU_WEEK"] >= 4, "AMT_REQ_CREDIT_BUREAU_WEEK"] = 4
application.loc[application["AMT_REQ_CREDIT_BUREAU_MON"] >= 10, "AMT_REQ_CREDIT_BUREAU_MON"] = 10
application.loc[application["AMT_REQ_CREDIT_BUREAU_QRT"] >= 5, "AMT_REQ_CREDIT_BUREAU_QRT"] = 5
application.loc[application["AMT_REQ_CREDIT_BUREAU_YEAR"] >= 10, "AMT_REQ_CREDIT_BUREAU_YEAR"] = 10

# Замена аномальных наблюдений непрерывными признаками
application.loc[application["AMT_INCOME_TOTAL"] >= 5000000, "AMT_INCOME_TOTAL"] = 5000000

In [ ]:
# Создание новых признаков для application

# Credit to total income ratio (Отношение кредита к общему доходу)
application["CREDIT_INCOME_RATIO"] = application["AMT_CREDIT"]/application["AMT_INCOME_TOTAL"]
add_description("application_{train|test}.csv", "CREDIT_INCOME_RATIO", "Credit to total income ratio")

# Annuity to total income ratio (Отношение аннуитета к общему доходу)
application["ANNUITY_INCOME_RATIO"] = application["AMT_ANNUITY"]/application["AMT_INCOME_TOTAL"]
add_description("application_{train|test}.csv", "ANNUITY_INCOME_RATIO", "Annuity to total income ratio")

# Credit to annuity ratio (indicative of credit duration) (Отношение кредита к аннуитету (с указанием продолжительности кредита))
application["CREDIT_ANNUITY_RATIO"] = application["AMT_CREDIT"]/application["AMT_ANNUITY"]
add_description("application_{train|test}.csv", "CREDIT_ANNUITY_RATIO", "Credit to annuity ratio (indicative of credit duration)")

# Estimated age at the end of the credit (Предполагаемый возраст в конце кредита)
application["AGE_END_CREDIT"] = -application["DAYS_BIRTH"]/365.25 + application["CREDIT_ANNUITY_RATIO"]
add_description("application_{train|test}.csv", "AGE_END_CREDIT", "Estimated age at the end of the credit")

# Personal contribution for purchasing goods (good price minus credit) (Личный взнос на покупку товара (хорошая цена минус кредит))
application["PERSONAL_CONTRIBUTION"] = (application["AMT_GOODS_PRICE"] - application["AMT_CREDIT"])
add_description("application_{train|test}.csv", "PERSONAL_CONTRIBUTION", "Personal contribution for purchasing goods (good price minus credit)")

# Average credit score (Средний кредитный рейтинг)
application["AVG_EXT_SOURCE"] = application[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
add_description("application_{train|test}.csv", "AVG_EXT_SOURCE", "Average credit score")

# Standard deviation between credit scores (Стандартное отклонение между кредитными баллами)
application['NEW_SCORES_STD'] = application[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
add_description("application_{train|test}.csv", "NEW_SCORES_STD", "Standard deviation between credit scores")

# Fraction of life employed (Доля занятой жизни)
application['EMPLOY_BIRTH_RATIO'] = application['DAYS_EMPLOYED'] / application['DAYS_BIRTH']
add_description("application_{train|test}.csv", "EMPLOY_BIRTH_RATIO", "Fraction of life employed")

# Fraction of life with current car (Доля жизни с текущим автомобилем)
application['CAR_BIRTH_RATIO'] = application['OWN_CAR_AGE'] / application['DAYS_BIRTH']
add_description("application_{train|test}.csv", "CAR_BIRTH_RATIO", "Fraction of life with current car")

# Fraction of life with current phone (Доля жизни с текущим телефоном)
application['PHONE_BIRTH_RATIO'] = application['DAYS_LAST_PHONE_CHANGE'] / application['DAYS_BIRTH']
add_description("application_{train|test}.csv", "PHONE_BIRTH_RATIO", "Fraction of life with current phone");


In [ ]:
# Проверка

application.shape

In [ ]:
# Сохранение application

application.to_csv("application.csv", index=False)

In [ ]:
# Сохранение homecredit_columns_description

homecredit_columns_description.to_csv("homecredit_columns_description.csv", index=False)